## *This is the most up-to-date plotting script*

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import mplhep as hep
hep.style.use("CMS")
from coffea import util
import itertools
import os, sys
import glob
import copy
from scipy.optimize import curve_fit
import uproot

sys.path.append('../python/')
from functions import loadCoffeaFile, getLabelMap, getCoffeaFilenames, plotBackgroundEstimate, getHist, lumi


In [ ]:
# make image save file directories #

In [ ]:
directories = [
    'images/png/closureTest/2016all',
    'images/png/closureTest/2016APV',
    'images/png/closureTest/2016',
    'images/png/closureTest/2017',
    'images/png/closureTest/2018',
    'images/pdf/closureTest/2016all',
    'images/pdf/closureTest/2016APV',
    'images/pdf/closureTest/2016',
    'images/pdf/closureTest/2017',
    'images/pdf/closureTest/2018',
    
    'images/png/ttbarmass/2016all',
    'images/png/ttbarmass/2016APV',
    'images/png/ttbarmass/2016',
    'images/png/ttbarmass/2017',
    'images/png/ttbarmass/2018',
    'images/pdf/ttbarmass/2016all',
    'images/pdf/ttbarmass/2016APV',
    'images/pdf/ttbarmass/2016',
    'images/pdf/ttbarmass/2017',
    'images/pdf/ttbarmass/2018',
]


for path in directories:
    if not os.path.exists(path):
        os.makedirs(path)

## analysis categories

In [ ]:
IOV = '2016'

# analysis categories #

label_dict = loadCoffeaFile()['analysisCategories']

label_to_int_dict = {label: i for i, label in label_dict.items()}

signal_cats = [ i for label, i in label_to_int_dict.items() if '2t' in label]
pretag_cats = [ i for label, i in label_to_int_dict.items() if 'pre' in label]
antitag_cats = [ i for label, i in label_to_int_dict.items() if 'at' in label]


## scale factors

In [ ]:
IOVs = ['2016']

lumi = {
    "2016APV": 19800.,
    "2016": 16120., #35920 - 19800
    "2016all": 35920,
    "2017": 41530.,
    "2018": 59740.
}

t_BR = 0.6741
ttbar_BR = 0.4544 #PDG 2019
ttbar_xs1 = 831.76 * (0.09210) #pb For ttbar mass from 700 to 1000
ttbar_xs2 = 831.76 * (0.02474) #pb For ttbar mass from 1000 to Inf
toptag_sf = 0.9
toptag_kf = 1.0 #0.7
qcd_xs = 13700000.0 #pb From https://cms-gen-dev.cern.ch/xsdb

zprime_xs = {
    '1000': 2.222,
    '1500': 0.387,
    '2000': 0.09428,
    '2500': 0.0279,
    '3000': 0.009327,
    '3500': 0.003507,
    '4000': 0.001484,
    '4500': 0.0007087,
    '5000': 0.0003801,
}



RSGluon_xs = {
    '1000': 21.03,
    '1500': 3.656,
    '2000': 0.9417,
    '2500': 0.3039,
    '3000': 0.1163,
    '3500': 0.05138,
    '4000': 0.02556,
    '4500': 0.01422,
    '5000': 0.008631,
}






# transfer function

In [ ]:
# fit N events from CR to N events from SR


# loose to fail | pretag to antitag

def rlf(x, mt, p0, p1):
    
    return x*(p0 + p1 * mt)


# pass to loose | signal to pretag

def rtl(x, mt, mtt, p2, p3, p4, p5):
    
    return x * (p2 + p3 * mt + p4 * mt * mt) * (p5 * mtt) 
    

In [ ]:
mtt_vs_mt = getHist('mtt_vs_mt', 'JetHT', False, '2016', sum_axes=[], integrate_axes={'systematic':'nominal'})
tt_mtt_vs_mt = getHist('mtt_vs_mt', 'TTbar', False, '2016', sum_axes=[], integrate_axes={'systematic':'nominal'})


# get 2d histograms of mt and mtt
h2dsr = mtt_vs_mt[{'anacat':signal_cats}][{'anacat':sum}]
h2dcr = mtt_vs_mt[{'anacat':antitag_cats}][{'anacat':sum}]
h2dpr = mtt_vs_mt[{'anacat':pretag_cats}][{'anacat':sum}]


# N events in signal, antitag (control), and pretag region
Nsr = h2dsr.values()
Ncr = h2dcr.values()
Npr = h2dpr.values()


# 2d array of mt and mtt bins
bins_mt = h2dsr.axes['jetmass'].edges[:-1]
bins_mtt = h2dsr.axes['ttbarmass'].edges[:-1]
mt, mtt = np.meshgrid(bins_mt, bins_mtt)

# flatten arrays for fitting
f_mt  =  mt.flatten()
f_mtt = mtt.flatten()
f_nsr = Nsr.flatten()
f_ncr = Ncr.flatten()
f_npr = Npr.flatten()


## fit the transfer functions

In [ ]:
# loose to fail | pretag to antitag


y = np.where(f_ncr > 0, f_npr/f_ncr, 0)

popt, pcov = curve_fit(lambda x, p0, p1: rlf(x, f_mt, p0, p1), 
                       f_ncr, 
                       f_npr, 
                       p0=[1,1])


p0, p1 = popt

f_rlf = rlf(np.ones_like(Ncr), mt, p0, p1)



# pass to loose | signal to pretag


y = np.where(f_npr > 0, f_nsr/f_npr, 0)





popt, pcov = curve_fit(lambda x, p2, p3, p4, p5: rtl(x, f_mt, f_mtt, p2, p3, p4, p5), 
                       f_npr,
                       f_nsr,
                       p0=[1,1,1,1], bounds=[-10,10])


p2, p3, p4, p5 = popt

f_rtl = rtl(np.ones_like(Ncr), mt, mtt, p2, p3, p4, p5)


## fit the transfer function by analysis category

In [ ]:
transfer_functions = {}
cats = ['0bcen', '0bfwd', '1bcen', '1bfwd', '2bcen', '2bfwd']
cat_labels = ['cen0b', 'fwd0b', 'cen1b', 'fwd1b', 'cen2b', 'fwd2b']

mtt_vs_mt = getHist('mtt_vs_mt', 'JetHT', False, '2016', sum_axes=[], integrate_axes={})


for cat in cats:

    signal_cat  = label_to_int_dict['2t'+cat]
    pretag_cat  = label_to_int_dict['pret'+cat]
    antitag_cat = label_to_int_dict['at'+cat]


    # get 2d histograms of mt and mtt
    h2dsr = mtt_vs_mt[{'anacat':signal_cat, 'systematic':'nominal'}]
    h2dcr = mtt_vs_mt[{'anacat':antitag_cat, 'systematic':'nominal'}]
    h2dpr = mtt_vs_mt[{'anacat':pretag_cat, 'systematic':'nominal'}]


    # N events in signal, antitag (control), and pretag region
    Nsr = h2dsr.values()
    Ncr = h2dcr.values()
    Npr = h2dpr.values()


    # 2d array of mt and mtt bins
    bins_mt = h2dsr.axes['jetmass'].edges[:-1]
    bins_mtt = h2dsr.axes['ttbarmass'].edges[:-1]
    mtt, mt = np.meshgrid(bins_mtt, bins_mt)

    # flatten arrays for fitting
    f_mt  =  mt.flatten()
    f_mtt = mtt.flatten()
    f_nsr = Nsr.flatten()
    f_ncr = Ncr.flatten()
    f_npr = Npr.flatten()
    
    
    
    
    
    # loose to fail | pretag to antitag


    y = np.where(f_ncr > 0, f_npr/f_ncr, 0)

    popt, pcov = curve_fit(lambda x, p0, p1: rlf(x, f_mt, p0, p1), 
                           f_ncr, 
                           f_npr, 
                           p0=[1,1])


    p0, p1 = popt

    f_rlf = rlf(np.ones_like(Ncr), mt, p0, p1)



    # pass to loose | signal to pretag


    y = np.where(f_npr > 0, f_nsr/f_npr, 0)





    popt, pcov = curve_fit(lambda x, p2, p3, p4, p5: rtl(x, f_mt, f_mtt, p2, p3, p4, p5), 
                           f_npr,
                           f_nsr,
                           p0=[1,1,1,1], bounds=[-10,10])


    p2, p3, p4, p5 = popt

    f_rtl = rtl(np.ones_like(Ncr), mt, mtt, p2, p3, p4, p5)
    
    
    
    transfer_functions[cat] = f_rlf


## plot data and background

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=2, height_ratios=[3, 1])

IOV = '2016'

httbar = getHist('ttbarmass', 'TTbar', False, IOV, sum_axes=['anacat'], integrate_axes={'anacat':signal_cats,'systematic':'nominal'})
hdata  = getHist('ttbarmass', 'JetHT', False, IOV, sum_axes=['anacat'], integrate_axes={'anacat':signal_cats,'systematic':'nominal'})


httbar = getHist('mtt_vs_mt', 'TTbar', False, IOV,
                 sum_axes=['anacat'],
                 integrate_axes={'anacat':signal_cats,'systematic':'nominal'})[{'jetmass':sum}]
hdata = getHist('mtt_vs_mt', 'JetHT', False, IOV,
                 sum_axes=['anacat'],
                 integrate_axes={'anacat':signal_cats,'systematic':'nominal'})[{'jetmass':sum}]



h_mtt_mt = getHist('mtt_vs_mt', 'JetHT', False, IOV, sum_axes=['anacat'],
                   integrate_axes={'anacat':signal_cats,'systematic':'nominal'})#

h_mtt_mt = h_mtt_mt
hntmj_2d = h_mtt_mt#*f_rtl
hntmj = hntmj_2d[{'jetmass':sum}]

hbkg = hntmj + httbar


# hep.histplot(hdata, histtype='errorbar', color='k', ax=ax1, label='Data')
hep.histplot(hbkg, histtype='fill', color='xkcd:pale gold', label='NTMJ Bkg Est', ax=ax1)
hep.histplot(httbar, histtype='fill', color='xkcd:deep red', label='TTbar', ax=ax1)


text = 'Preliminary'+'\n'+r'$\Delta y$ inclusive'+'\n'+r'b tag inclusive'
hep.cms.label('', data=True, lumi='{0:0.1f}'.format(lumi['2016']/1000.), year='2016', loc=2, fontsize=20, ax=ax1)
hep.cms.text(text, loc=2, fontsize=20, ax=ax1)




ax1.set_yscale('log')
ax1.set_ylim(1e-2,1e7)

ax1.set_ylabel('Events')

ratio_plot = hdata / hbkg.values()


hep.histplot(ratio_plot, ax=ax2, histtype='errorbar', color='black')
ax2.set_ylim(0,2)
ax2.axhline(1, color='black', ls='--')
ax2.set_ylabel('Data/Bkg')
ax2.set_xlabel(ax1.get_xlabel())
ax1.set_xlabel('')


ax1.legend(loc=1, fontsize=15)


imagefile = f'images/png/closureTest/{IOV}/closuretest_inclusive.png'

plt.savefig(imagefile)
plt.savefig(imagefile.replace('png','pdf'))
print('saving ', imagefile)
print('saving ', imagefile.replace('png','pdf'))



## mass window 100 to 200 GeV

In [ ]:
qcdfile = util.load('../outputs/QCD_2016_test.coffea')
qcd_sf = lumi['2016'] * qcd_xs / qcdfile['cutflow']['sumw']
qcd_sf

In [ ]:
qcdfile['cutflow']['all events']

In [ ]:
# fig, (ax1, ax2) = plt.subplots(nrows=2, height_ratios=[3, 1])


fig, ax1 = plt.subplots()#nrows=2, height_ratios=[3, 1])

IOV = '2016'



httbar2d = getHist('mtt_vs_mt', 'TTbar', False, IOV,
                 sum_axes=['anacat'],
                 integrate_axes={'anacat':signal_cats,'systematic':'nominal'})


hqcd2d = qcdfile['mtt_vs_mt'][{'anacat':signal_cats, 'systematic':'nominal'}][{'anacat':sum}]

httbar = httbar2d[100j, :] + httbar2d[125j, :] + httbar2d[150j, :] + httbar2d[200j, :]
hdata  = hdata2d[100j, :] + hdata2d[125j, :] + hdata2d[150j, :] + hdata2d[200j, :]


hqcd  = (httbar2d[4, :] + httbar2d[5, :] + httbar2d[6, :] + httbar2d[7, :])
hqcd  = (hqcd2d[4, :] + hqcd2d[5, :] + hqcd2d[6, :] + hqcd2d[7, :])*qcd_sf


h_mtt_mt = getHist('mtt_vs_mt', 'JetHT', False, IOV, sum_axes=['anacat'],
                   integrate_axes={'anacat':signal_cats,'systematic':'nominal'})#

hntmj = hqcd #h_mtt_mt[100j, :] + h_mtt_mt[125j, :] + h_mtt_mt[150j, :] + h_mtt_mt[200j, :]

hbkg = hntmj + httbar


# hep.histplot(hdata, histtype='errorbar', color='k', ax=ax1, label='Data')
hep.histplot(hqcd+httbar, histtype='fill', color='xkcd:pale gold', label='QCD', ax=ax1)
hep.histplot(httbar, histtype='fill', color='xkcd:deep red', label='TTbar', ax=ax1)


text = 'Preliminary'+'\n'+r'$\Delta y$ inclusive'+', '+r'b tag inclusive' + '\n' + r'100 < $m_{t}$ < 200 GeV'
hep.cms.label('', data=True, lumi='{0:0.1f}'.format(lumi['2016']/1000.), year='2016', loc=2, fontsize=20, ax=ax1)
hep.cms.text(text, loc=2, fontsize=20, ax=ax1)




ax1.set_yscale('log')
ax1.set_ylim(1e-2,1e7)

ax1.set_ylabel('Events')

ratio_plot = hdata / hbkg.values()


# hep.histplot(ratio_plot, ax=ax2, histtype='errorbar', color='black')
# ax2.set_ylim(0,2)
# ax2.axhline(1, color='black', ls='--')
# ax2.set_ylabel('Data/Bkg')
# ax2.set_xlabel(ax1.get_xlabel())
# ax1.set_xlabel('')


ax1.legend(loc=1, fontsize=15)


imagefile = f'images/png/closureTest/{IOV}/closuretest_inclusive.png'

plt.savefig(imagefile)
plt.savefig(imagefile.replace('png','pdf'))
print('saving ', imagefile)
print('saving ', imagefile.replace('png','pdf'))



In [ ]:
hqcd2d.axes['jetmass'].centers[4]

In [ ]:
hqcd2d[100j:200j, :]

In [ ]:
qcd_sf = lumi['2016'] * qcd_xs / qcdfile['cutflow']['sumw']
qcd_sf

In [ ]:
hqcd2d = qcdfile['mtt_vs_mt'][{'anacat':signal_cats, 'systematic':'nominal'}][{'anacat':sum}]

### get signal files

In [ ]:
RSGluon1000file = util.load('../outputs/RSGluon1000_2016.coffea')
RSGluon1500file = util.load('../outputs/RSGluon1500_2016.coffea')
RSGluon2000file = util.load('../outputs/RSGluon2000_2016.coffea')
RSGluon2500file = util.load('../outputs/RSGluon2500_2016.coffea')
RSGluon3000file = util.load('../outputs/RSGluon3000_2016.coffea')
RSGluon3500file = util.load('../outputs/RSGluon3500_2016.coffea')
RSGluon4000file = util.load('../outputs/RSGluon4000_2016.coffea')
RSGluon4500file = util.load('../outputs/RSGluon4500_2016.coffea')
RSGluon5000file = util.load('../outputs/RSGluon5000_2016.coffea')

## plot $m_{t\bar{t}}$

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=2, height_ratios=[3, 1])



IOV = '2016'

httbar = getHist('ttbarmass', 'TTbar', False, IOV, sum_axes=['anacat'], integrate_axes={'anacat':signal_cats,'systematic':'nominal'})
hdata  = getHist('ttbarmass', 'JetHT', False, IOV, sum_axes=['anacat'], integrate_axes={'anacat':signal_cats,'systematic':'nominal'})


httbar = getHist('mtt_vs_mt', 'TTbar', False, IOV,
                 sum_axes=['anacat'],
                 integrate_axes={'anacat':signal_cats,'systematic':'nominal'})[{'jetmass':sum}]
hdata = getHist('mtt_vs_mt', 'JetHT', False, IOV,
                 sum_axes=['anacat'],
                 integrate_axes={'anacat':signal_cats,'systematic':'nominal'})[{'jetmass':sum}]



h_mtt_mt = getHist('mtt_vs_mt', 'JetHT', False, IOV, 
                   sum_axes=['anacat'], 
                   integrate_axes={'anacat':pretag_cats, 'systematic':'nominal'})



hntmj_2d = h_mtt_mt * f_rlf # * f_rtl
hntmj = hntmj_2d[{'jetmass':sum}]


hbkg = hntmj + httbar





hRSGluon1000 = RSGluon1000file['mtt_vs_mt'][{'anacat':signal_cats}][{'anacat':sum,'systematic':sum}][{'jetmass':sum}] * lumi['2016'] * RSGluon_xs['1000'] / RSGluon1000file['cutflow']['sumw']
hRSGluon1500 = RSGluon1500file['mtt_vs_mt'][{'anacat':signal_cats}][{'anacat':sum,'systematic':sum}][{'jetmass':sum}] * lumi['2016'] * RSGluon_xs['1500'] / RSGluon1500file['cutflow']['sumw']
hRSGluon2000 = RSGluon2000file['mtt_vs_mt'][{'anacat':signal_cats}][{'anacat':sum,'systematic':sum}][{'jetmass':sum}] * lumi['2016'] * RSGluon_xs['2000'] / RSGluon2000file['cutflow']['sumw']
hRSGluon2500 = RSGluon2500file['mtt_vs_mt'][{'anacat':signal_cats}][{'anacat':sum,'systematic':sum}][{'jetmass':sum}] * lumi['2016'] * RSGluon_xs['2500'] / RSGluon2500file['cutflow']['sumw']
hRSGluon3000 = RSGluon3000file['mtt_vs_mt'][{'anacat':signal_cats}][{'anacat':sum,'systematic':sum}][{'jetmass':sum}] * lumi['2016'] * RSGluon_xs['3000'] / RSGluon3000file['cutflow']['sumw']
hRSGluon3500 = RSGluon3500file['mtt_vs_mt'][{'anacat':signal_cats}][{'anacat':sum,'systematic':sum}][{'jetmass':sum}] * lumi['2016'] * RSGluon_xs['3500'] / RSGluon3500file['cutflow']['sumw']
hRSGluon4000 = RSGluon4000file['mtt_vs_mt'][{'anacat':signal_cats}][{'anacat':sum,'systematic':sum}][{'jetmass':sum}] * lumi['2016'] * RSGluon_xs['4000'] / RSGluon4000file['cutflow']['sumw']
hRSGluon4500 = RSGluon4500file['mtt_vs_mt'][{'anacat':signal_cats}][{'anacat':sum,'systematic':sum}][{'jetmass':sum}] * lumi['2016'] * RSGluon_xs['4500'] / RSGluon4500file['cutflow']['sumw']
hRSGluon5000 = RSGluon5000file['mtt_vs_mt'][{'anacat':signal_cats}][{'anacat':sum,'systematic':sum}][{'jetmass':sum}] * lumi['2016'] * RSGluon_xs['5000'] / RSGluon5000file['cutflow']['sumw']



hep.histplot(hdata, histtype='errorbar', color='k', ax=ax1, label='Data')
hep.histplot(hbkg, histtype='fill', color='xkcd:pale gold', label='NTMJ Bkg Est', ax=ax1)
hep.histplot(httbar, histtype='fill', color='xkcd:deep red', label='TTbar', ax=ax1)
hep.histplot(hRSGluon1000, histtype='step', color='C0', label='RSGluon 1 TeV', ax=ax1)
hep.histplot(hRSGluon1500, histtype='step', color='C1', label='RSGluon 1.5 TeV', ax=ax1)
hep.histplot(hRSGluon2000, histtype='step', color='C2', label='RSGluon 2 TeV', ax=ax1)
hep.histplot(hRSGluon2500, histtype='step', color='C3', label='RSGluon 2.5 TeV', ax=ax1)
hep.histplot(hRSGluon3000, histtype='step', color='C4', label='RSGluon 3 TeV', ax=ax1)
hep.histplot(hRSGluon3500, histtype='step', color='C5', label='RSGluon 3.5 TeV', ax=ax1)
hep.histplot(hRSGluon4000, histtype='step', color='C6', label='RSGluon 4 TeV', ax=ax1)
hep.histplot(hRSGluon4500, histtype='step', color='C7', label='RSGluon 4,5 TeV', ax=ax1)
hep.histplot(hRSGluon5000, histtype='step', color='C8', label='RSGluon 5 TeV', ax=ax1)





ax1.set_yscale('log')
ax1.set_ylim(1e-2,1e7)

ax1.set_ylabel('Events')

ratio_plot = hdata / hbkg.values()



text = 'Preliminary'+'\n'+r'$\Delta y$ inclusive'+'\n'+r'b tag inclusive'
hep.cms.label('', data=True, lumi='{0:0.1f}'.format(lumi['2016']/1000.), year='2016', loc=2, fontsize=20, ax=ax1)
hep.cms.text(text, loc=2, fontsize=20, ax=ax1)



hep.histplot(ratio_plot, ax=ax2, histtype='errorbar', color='black')
ax2.set_ylim(0,2)
ax2.axhline(1, color='black', ls='--')
ax2.set_ylabel('Data/Bkg')
ax2.set_xlabel(ax1.get_xlabel())
ax1.set_xlabel('')


ax1.legend(loc=1, fontsize=14)


imagefile = f'images/png/ttbarmass/{IOV}/ttbarmass_inclusive.png'

plt.savefig(imagefile)
plt.savefig(imagefile.replace('png','pdf'))
print('saving ', imagefile)
print('saving ', imagefile.replace('png','pdf'))

## loop through analysis categories and systematics

In [ ]:
# cats = ['0bcen', '0bfwd', '1bcen', '1bfwd', '2bcen', '2bfwd']
# cat_labels = ['cen0b', 'fwd0b', 'cen1b', 'fwd1b', 'cen2b', 'fwd2b']
# systematics = ['nominal', 'pileup', 'prefiring', 'pdf']
# syst_labels = ['nominal']
# for s in systematics:
#     if not 'nominal' in s:
#         syst_labels.append(s+'Down')
#         syst_labels.append(s+'Up')

        
        
        
cats = ['0bcen']
cat_labels = ['cen0b']
systematics = ['nominal','pileup','pdf','jes','jer']
syst_labels = ['nominal']
for s in systematics:
    if not 'nominal' in s:
        syst_labels.append(s+'Down')
        syst_labels.append(s+'Up')
#         
        
cats = ['0bcen', '0bfwd', '1bcen', '1bfwd', '2bcen', '2bfwd']
cat_labels = ['cen0b', 'fwd0b', 'cen1b', 'fwd1b', 'cen2b', 'fwd2b']

## make root files for 2DAlphabet

In [ ]:
fdata = uproot.recreate('TTbarAllHad16_Data.root')
fttbar = uproot.recreate('TTbarAllHad16_TTbar.root')
f1000 = uproot.recreate('TTbarAllHad16_signalRSGluon1000.root')
f1500 = uproot.recreate('TTbarAllHad16_signalRSGluon1500.root')
f2000 = uproot.recreate('TTbarAllHad16_signalRSGluon2000.root')
f2500 = uproot.recreate('TTbarAllHad16_signalRSGluon2500.root')
f3000 = uproot.recreate('TTbarAllHad16_signalRSGluon3000.root')
f3500 = uproot.recreate('TTbarAllHad16_signalRSGluon3500.root')
f4000 = uproot.recreate('TTbarAllHad16_signalRSGluon4000.root')
f4500 = uproot.recreate('TTbarAllHad16_signalRSGluon4500.root')
f5000 = uproot.recreate('TTbarAllHad16_signalRSGluon5000.root')

cats = ['test']
cat_labels = ['']


for cat, catname in zip(cats, cat_labels):
    
#     signal_cats  = label_to_int_dict['2t'+cat]
#     pretag_cats  = label_to_int_dict['pret'+cat]
#     antitag_cats = label_to_int_dict['at'+cat]


    
    
    for syst in syst_labels:
        
        print(cat, syst)
        
     
        hRSGluon1000_pass = RSGluon1000file['mtt_vs_mt'][{'anacat':signal_cats,'systematic':syst}][{'anacat':sum}] * lumi['2016'] * RSGluon_xs['1000'] / RSGluon1000file['cutflow']['sumw']
        hRSGluon1500_pass = RSGluon1500file['mtt_vs_mt'][{'anacat':signal_cats,'systematic':syst}][{'anacat':sum}] * lumi['2016'] * RSGluon_xs['1500'] / RSGluon1500file['cutflow']['sumw']
        hRSGluon2000_pass = RSGluon2000file['mtt_vs_mt'][{'anacat':signal_cats,'systematic':syst}][{'anacat':sum}] * lumi['2016'] * RSGluon_xs['2000'] / RSGluon2000file['cutflow']['sumw']
        hRSGluon2500_pass = RSGluon2500file['mtt_vs_mt'][{'anacat':signal_cats,'systematic':syst}][{'anacat':sum}] * lumi['2016'] * RSGluon_xs['2500'] / RSGluon2500file['cutflow']['sumw']
        hRSGluon3000_pass = RSGluon3000file['mtt_vs_mt'][{'anacat':signal_cats,'systematic':syst}][{'anacat':sum}] * lumi['2016'] * RSGluon_xs['3000'] / RSGluon3000file['cutflow']['sumw']
        hRSGluon3500_pass = RSGluon3500file['mtt_vs_mt'][{'anacat':signal_cats,'systematic':syst}][{'anacat':sum}] * lumi['2016'] * RSGluon_xs['3500'] / RSGluon3500file['cutflow']['sumw']
        hRSGluon4000_pass = RSGluon4000file['mtt_vs_mt'][{'anacat':signal_cats,'systematic':syst}][{'anacat':sum}] * lumi['2016'] * RSGluon_xs['4000'] / RSGluon4000file['cutflow']['sumw']
        hRSGluon4500_pass = RSGluon4500file['mtt_vs_mt'][{'anacat':signal_cats,'systematic':syst}][{'anacat':sum}] * lumi['2016'] * RSGluon_xs['4500'] / RSGluon4500file['cutflow']['sumw']
        hRSGluon5000_pass = RSGluon5000file['mtt_vs_mt'][{'anacat':signal_cats,'systematic':syst}][{'anacat':sum}] * lumi['2016'] * RSGluon_xs['5000'] / RSGluon5000file['cutflow']['sumw']

        hRSGluon1000_fail = RSGluon1000file['mtt_vs_mt'][{'anacat':antitag_cats,'systematic':syst}][{'anacat':sum}] * lumi['2016'] * RSGluon_xs['1000'] / RSGluon1000file['cutflow']['sumw']
        hRSGluon1500_fail = RSGluon1500file['mtt_vs_mt'][{'anacat':antitag_cats,'systematic':syst}][{'anacat':sum}] * lumi['2016'] * RSGluon_xs['1500'] / RSGluon1500file['cutflow']['sumw']
        hRSGluon2000_fail = RSGluon2000file['mtt_vs_mt'][{'anacat':antitag_cats,'systematic':syst}][{'anacat':sum}] * lumi['2016'] * RSGluon_xs['2000'] / RSGluon2000file['cutflow']['sumw']
        hRSGluon2500_fail = RSGluon2500file['mtt_vs_mt'][{'anacat':antitag_cats,'systematic':syst}][{'anacat':sum}] * lumi['2016'] * RSGluon_xs['2500'] / RSGluon2500file['cutflow']['sumw']
        hRSGluon3000_fail = RSGluon3000file['mtt_vs_mt'][{'anacat':antitag_cats,'systematic':syst}][{'anacat':sum}] * lumi['2016'] * RSGluon_xs['3000'] / RSGluon3000file['cutflow']['sumw']
        hRSGluon3500_fail = RSGluon3500file['mtt_vs_mt'][{'anacat':antitag_cats,'systematic':syst}][{'anacat':sum}] * lumi['2016'] * RSGluon_xs['3500'] / RSGluon3500file['cutflow']['sumw']
        hRSGluon4000_fail = RSGluon4000file['mtt_vs_mt'][{'anacat':antitag_cats,'systematic':syst}][{'anacat':sum}] * lumi['2016'] * RSGluon_xs['4000'] / RSGluon4000file['cutflow']['sumw']
        hRSGluon4500_fail = RSGluon4500file['mtt_vs_mt'][{'anacat':antitag_cats,'systematic':syst}][{'anacat':sum}] * lumi['2016'] * RSGluon_xs['4500'] / RSGluon4500file['cutflow']['sumw']
        hRSGluon5000_fail = RSGluon5000file['mtt_vs_mt'][{'anacat':antitag_cats,'systematic':syst}][{'anacat':sum}] * lumi['2016'] * RSGluon_xs['5000'] / RSGluon5000file['cutflow']['sumw']

        httbar_pass = getHist('mtt_vs_mt', 'TTbar', False, IOV, sum_axes=['anacat'], integrate_axes={'anacat':signal_cats, 'systematic':syst})
        httbar_fail = getHist('mtt_vs_mt', 'TTbar', False, IOV, sum_axes=['anacat'], integrate_axes={'anacat':antitag_cats, 'systematic':syst})
        
        
            
        
        
        
        if 'nominal' in syst:
            
            hdata_pass  = getHist('mtt_vs_mt', 'JetHT', False, IOV, sum_axes=['anacat'], integrate_axes={'anacat':signal_cats, 'systematic':'nominal'})
            hdata_fail  = getHist('mtt_vs_mt', 'JetHT', False, IOV, sum_axes=['anacat'], integrate_axes={'anacat':antitag_cats, 'systematic':'nominal'})
        
            fdata["MttvsMtPass"] = hdata_pass
            fdata["MttvsMtFail"] = hdata_fail
            
            fttbar["MttvsMtPass"] = httbar_pass
            fttbar["MttvsMtFail"] = httbar_fail
            
            
            f1000["MttvsMtPass"] = hRSGluon1000_pass
            f1500["MttvsMtPass"] = hRSGluon1500_pass
            f2000["MttvsMtPass"] = hRSGluon2000_pass
            f2500["MttvsMtPass"] = hRSGluon2500_pass
            f3000["MttvsMtPass"] = hRSGluon3000_pass
            f3500["MttvsMtPass"] = hRSGluon3500_pass
            f4000["MttvsMtPass"] = hRSGluon4000_pass
            f4500["MttvsMtPass"] = hRSGluon4500_pass
            f5000["MttvsMtPass"] = hRSGluon5000_pass
            
            f1000["MttvsMtFail"] = hRSGluon1000_fail
            f1500["MttvsMtFail"] = hRSGluon1500_fail
            f2000["MttvsMtFail"] = hRSGluon2000_fail
            f2500["MttvsMtFail"] = hRSGluon2500_fail
            f3000["MttvsMtFail"] = hRSGluon3000_fail
            f3500["MttvsMtFail"] = hRSGluon3500_fail
            f4000["MttvsMtFail"] = hRSGluon4000_fail
            f4500["MttvsMtFail"] = hRSGluon4500_fail
            f5000["MttvsMtFail"] = hRSGluon5000_fail
            
        else:
                        
            systname = syst.upper()[:-2] + 'up' if 'Up' in syst else syst.upper()[:-4] + 'down'

            
#             fdata["MttvsMtPass"+systname] = hdata_pass
#             fdata["MttvsMtFail"+systname] = hdata_fail

            fttbar["MttvsMtPass"+systname] = httbar_pass
            fttbar["MttvsMtFail"+systname] = httbar_fail
        
            f1000["MttvsMtPass"+systname] = hRSGluon1000_pass
            f1500["MttvsMtPass"+systname] = hRSGluon1500_pass
            f2000["MttvsMtPass"+systname] = hRSGluon2000_pass
            f2500["MttvsMtPass"+systname] = hRSGluon2500_pass
            f3000["MttvsMtPass"+systname] = hRSGluon3000_pass
            f3500["MttvsMtPass"+systname] = hRSGluon3500_pass
            f4000["MttvsMtPass"+systname] = hRSGluon4000_pass
            f4500["MttvsMtPass"+systname] = hRSGluon4500_pass
            f5000["MttvsMtPass"+systname] = hRSGluon5000_pass
            
            f1000["MttvsMtFail"+systname] = hRSGluon1000_fail
            f1500["MttvsMtFail"+systname] = hRSGluon1500_fail
            f2000["MttvsMtFail"+systname] = hRSGluon2000_fail
            f2500["MttvsMtFail"+systname] = hRSGluon2500_fail
            f3000["MttvsMtFail"+systname] = hRSGluon3000_fail
            f3500["MttvsMtFail"+systname] = hRSGluon3500_fail
            f4000["MttvsMtFail"+systname] = hRSGluon4000_fail
            f4500["MttvsMtFail"+systname] = hRSGluon4500_fail
            f5000["MttvsMtFail"+systname] = hRSGluon5000_fail


                        
            
        
        

        
        
fdata.close()
fttbar.close()
f1000.close()
f1500.close()
f2000.close()
f2500.close()
f3000.close()
f3500.close()
f4000.close()
f4500.close()
f5000.close()

In [ ]:
lumi['2016'] * RSGluon_xs['2000'] / RSGluon2000file['cutflow']['sumw']

In [ ]:
httbar_pass = getHist('mtt_vs_mt', 'TTbar', False, IOV, sum_axes=['anacat'], integrate_axes={'anacat':signal_cats, 'systematic':'nominal'})


In [ ]:
print([int(b) for b in httbar_pass.axes['ttbarmass'].edges][:-1])

### closure test

In [ ]:
# f = uproot.recreate('regions.root')

# for cat, catname in zip(cats, cat_labels):
#     for syst in syst_labels:
        
        
#         fig, (ax1, ax2) = plt.subplots(nrows=2, height_ratios=[3, 1])


#         signal_cat = label_to_int_dict['2t'+cat]
#         pretag_cat = label_to_int_dict['pret'+cat]
        
        
        
#         httbar = getHist('ttbarmass', 'TTbar', False, IOV, 
#                          sum_axes=[], 
#                          integrate_axes={'anacat':signal_cat,
#                                          'systematic':syst,
#                                         }
#                         )
        
        
#         hdata = getHist('ttbarmass', 'JetHT', False, IOV, 
#                          sum_axes=[], 
#                          integrate_axes={'anacat':signal_cat,
#                                          'systematic':syst,
#                                         }
#                         )
#         h_mtt_mt = getHist('mtt_vs_mt', 'JetHT', False, IOV, 
#                            sum_axes=[], 
#                            integrate_axes={'anacat':pretag_cat,
#                                           'systematic':syst,
#                                           })
        
        
# #         hntmj_2d = h_mtt_mt * f_rlf
#         hntmj_2d = h_mtt_mt * transfer_functions[cat]
        
#         hntmj = hntmj_2d[{'jetmass':sum}]


#         hbkg = httbar + hntmj
        

#         hep.histplot(hdata, histtype='errorbar', color='k', ax=ax1, label='Data')
#         hep.histplot(hbkg, histtype='fill', color='xkcd:pale gold', label='NTMJ Bkg Est', ax=ax1)
#         hep.histplot(httbar, histtype='fill', color='xkcd:deep red', label='TTbar', ax=ax1)
    
        
#         ax1.set_ylim(1e-2,1e6)
#         ax1.set_yscale('log')
        
        
#         ratio_plot = hdata / hbkg.values()


#         hep.histplot(ratio_plot, ax=ax2, histtype='errorbar', color='black')
#         ax2.set_ylim(0,2)
#         ax2.axhline(1, color='black', ls='--')
#         ax2.set_ylabel('Data/Bkg')
#         ax2.set_xlabel(ax1.get_xlabel())
#         ax1.set_xlabel('')

        
#         dytext = ''
#         if 'cen' in cat:
#             dytext = r'$\Delta y$ < 1.0'
#         elif 'fwd' in cat:
#             dytext = r'$\Delta y$ > 1.0'

#         btext = ''
#         if '0b' in cat:
#             btext = '0 b-tags'
#         elif '1b' in cat:
#             btext = '1 b-tag'
#         elif '2b' in cat:
#             btext = '2 b-tags'
        
#         text = f'Preliminary\n{btext}, {dytext} \n'
#         hep.cms.label('', data=True, lumi='{0:0.1f}'.format(lumi['2016']/1000.), year='2016', loc=2, fontsize=20, ax=ax1)
#         hep.cms.text(text, loc=2, fontsize=20, ax=ax1)
        
        
#         ax1.legend(loc=1, fontsize=14)
        
#         plt.plot()
        
        
        
#         imagefile = f'images/png/closureTest/{IOV}/closuretest_{catname}_{syst}.png'

#         plt.savefig(imagefile)
#         plt.savefig(imagefile.replace('png','pdf'))
#         print('saving ', imagefile)
#         print('saving ', imagefile.replace('png','pdf'))


        
        
        
        

## plot ttbarmass histogram and save files 

In [ ]:
# f = uproot.recreate('regions.root')

# for cat, catname in zip(cats, cat_labels):
#     for syst in syst_labels:
        
        
#         fig, (ax1, ax2) = plt.subplots(nrows=2, height_ratios=[3, 1])


#         signal_cat = label_to_int_dict['2t'+cat]
#         pretag_cat = label_to_int_dict['pret'+cat]
        
        
        
#         httbar = getHist('ttbarmass', 'TTbar', False, IOV, 
#                          sum_axes=[], 
#                          integrate_axes={'anacat':signal_cat,
#                                          'systematic':syst,
#                                         }
#                         )
        
        
#         hdata = getHist('ttbarmass', 'JetHT', False, IOV, 
#                          sum_axes=[], 
#                          integrate_axes={'anacat':signal_cat,
#                                          'systematic':syst,
#                                         }
#                         )
#         h_mtt_mt = getHist('mtt_vs_mt', 'JetHT', False, IOV, sum_axes=['systematic'], integrate_axes={'anacat':pretag_cat})
        
        
#         hntmj_2d = h_mtt_mt * f_rlf * f_rtl
        
#         hntmj = hntmj_2d[{'jetmass':sum}]


#         hbkg = httbar + hntmj
        


#         hRSGluon1000 = RSGluon1000file['ttbarmass'][{'anacat':signal_cat,'systematic':syst}] * lumi['2016'] * RSGluon_xs['1000'] / RSGluon1000file['cutflow']['sumw']
#         hRSGluon1500 = RSGluon1500file['ttbarmass'][{'anacat':signal_cat,'systematic':syst}] * lumi['2016'] * RSGluon_xs['1500'] / RSGluon1500file['cutflow']['sumw']
#         hRSGluon2000 = RSGluon2000file['ttbarmass'][{'anacat':signal_cat,'systematic':syst}] * lumi['2016'] * RSGluon_xs['2000'] / RSGluon2000file['cutflow']['sumw']
#         hRSGluon2500 = RSGluon2500file['ttbarmass'][{'anacat':signal_cat,'systematic':syst}] * lumi['2016'] * RSGluon_xs['2500'] / RSGluon2500file['cutflow']['sumw']
#         hRSGluon3000 = RSGluon3000file['ttbarmass'][{'anacat':signal_cat,'systematic':syst}] * lumi['2016'] * RSGluon_xs['3000'] / RSGluon3000file['cutflow']['sumw']
#         hRSGluon3500 = RSGluon3500file['ttbarmass'][{'anacat':signal_cat,'systematic':syst}] * lumi['2016'] * RSGluon_xs['3500'] / RSGluon3500file['cutflow']['sumw']
#         hRSGluon4000 = RSGluon4000file['ttbarmass'][{'anacat':signal_cat,'systematic':syst}] * lumi['2016'] * RSGluon_xs['4000'] / RSGluon4000file['cutflow']['sumw']
#         hRSGluon4500 = RSGluon4500file['ttbarmass'][{'anacat':signal_cat,'systematic':syst}] * lumi['2016'] * RSGluon_xs['4500'] / RSGluon4500file['cutflow']['sumw']
#         hRSGluon5000 = RSGluon5000file['ttbarmass'][{'anacat':signal_cat,'systematic':syst}] * lumi['2016'] * RSGluon_xs['5000'] / RSGluon5000file['cutflow']['sumw']



#         hep.histplot(hdata, histtype='errorbar', color='k', ax=ax1, label='Data')
#         hep.histplot(hbkg, histtype='fill', color='xkcd:pale gold', label='NTMJ Bkg Est', ax=ax1)
#         hep.histplot(httbar, histtype='fill', color='xkcd:deep red', label='TTbar', ax=ax1)
#         hep.histplot(hRSGluon1000, histtype='step', color='C0', label='RSGluon 1 TeV', ax=ax1)
#         hep.histplot(hRSGluon1500, histtype='step', color='C1', label='RSGluon 1.5 TeV', ax=ax1)
#         hep.histplot(hRSGluon2000, histtype='step', color='C2', label='RSGluon 2 TeV', ax=ax1)
#         hep.histplot(hRSGluon2500, histtype='step', color='C3', label='RSGluon 2.5 TeV', ax=ax1)
#         hep.histplot(hRSGluon3000, histtype='step', color='C4', label='RSGluon 3 TeV', ax=ax1)
#         hep.histplot(hRSGluon3500, histtype='step', color='C5', label='RSGluon 3.5 TeV', ax=ax1)
#         hep.histplot(hRSGluon4000, histtype='step', color='C6', label='RSGluon 4 TeV', ax=ax1)
#         hep.histplot(hRSGluon4500, histtype='step', color='C7', label='RSGluon 4,5 TeV', ax=ax1)
#         hep.histplot(hRSGluon5000, histtype='step', color='C8', label='RSGluon 5 TeV', ax=ax1)
    
        
#         ax1.set_ylim(1e-2,1e6)
#         ax1.set_yscale('log')
        
        
#         ratio_plot = hdata / hbkg.values()


#         hep.histplot(ratio_plot, ax=ax2, histtype='errorbar', color='black')
#         ax2.set_ylim(0,2)
#         ax2.axhline(1, color='black', ls='--')
#         ax2.set_ylabel('Data/Bkg')
#         ax2.set_xlabel(ax1.get_xlabel())
#         ax1.set_xlabel('')

        
#         dytext = ''
#         if 'cen' in cat:
#             dytext = r'$\Delta y$ < 1.0'
#         elif 'fwd' in cat:
#             dytext = r'$\Delta y$ > 1.0'

#         btext = ''
#         if '0b' in cat:
#             btext = '0 b-tags'
#         elif '1b' in cat:
#             btext = '1 b-tag'
#         elif '2b' in cat:
#             btext = '2 b-tags'
        
#         text = f'Preliminary\n{btext}, {dytext} \n'
#         hep.cms.label('', data=True, lumi='{0:0.1f}'.format(lumi['2016']/1000.), year='2016', loc=2, fontsize=20, ax=ax1)
#         hep.cms.text(text, loc=2, fontsize=20, ax=ax1)
        
        
#         ax1.legend(loc=1, fontsize=14)
        
#         plt.plot()
        
        
        

#         f['data_obs'  + '_' + catname ] = hdata
#         f['ntmj'  + '_' + catname ] = httbar
#         f['ttbar' + '_' + catname] = hntmj
#         f['RSGluon1000' + '_' + catname ] = hRSGluon1000
#         f['RSGluon1500' + '_' + catname ] = hRSGluon1500
#         f['RSGluon2000' + '_' + catname ] = hRSGluon2000
#         f['RSGluon2500' + '_' + catname ] = hRSGluon2500
#         f['RSGluon3000' + '_' + catname ] = hRSGluon3000
#         f['RSGluon3500' + '_' + catname ] = hRSGluon3500
#         f['RSGluon4000' + '_' + catname ] = hRSGluon4000
#         f['RSGluon4500' + '_' + catname ] = hRSGluon4500
#         f['RSGluon5000' + '_' + catname ] = hRSGluon5000
        
#         imagefile = f'images/png/ttbarmass/{IOV}/ttbarmass_{catname}.png'

#         plt.savefig(imagefile)
#         plt.savefig(imagefile.replace('png','pdf'))
#         print('saving ', imagefile)
#         print('saving ', imagefile.replace('png','pdf'))

# f.close()

        
        
        
        

In [ ]:
f = uproot.open('all_plots.root')

In [ ]:
f.keys()

In [ ]:
hep.histplot(f['QCD_Fail_postfit_projy0;1'])

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=2, height_ratios=[3, 1])

IOV = '2016'



# hqcd = f['QCD_Fail_postfit_projy1;1']
hqcd = f['TotalBkg_Fail_postfit_projy1;1']

httbar = f['16_TTbar_Fail_postfit_projy1;1']
hdata = f['data_obs_Fail_postfit_projy1;1']


hep.histplot(hdata, histtype='errorbar', color='k', ax=ax1, label='Data')
hep.histplot(hqcd, histtype='fill', color='xkcd:pale gold', label='NTMJ Bkg Est', ax=ax1, stack=True)
hep.histplot(httbar, histtype='fill', color='xkcd:deep red', label='TTbar', ax=ax1)


text = 'Preliminary'+'\n'+r'$\Delta y$ inclusive'+', '+r'b tag inclusive' + '\n' + r'120 < $m_{t}$ < 220 GeV'
hep.cms.label('', data=True, lumi='{0:0.1f}'.format(lumi['2016']/1000.), year='2016', loc=2, fontsize=15, ax=ax1)
hep.cms.text(text, loc=2, fontsize=15, ax=ax1)




ax1.set_yscale('log')
ax1.set_ylim(1e-2,1e8)
ax1.set_xlim(800,3300)


ax1.set_ylabel('Events/bin', fontsize=20)

ratio_plot = hdata.values() / (hqcd.values() + httbar.values())

print(ratio_plot)

ax2.plot(hdata.axis().centers(), ratio_plot, color='k', ls='', marker='o')
ax2.set_ylim(0,2)
ax2.axhline(1, color='black', ls='--')
ax2.set_ylabel('Data/Bkg', fontsize=20)
ax2.set_xlabel(ax1.get_xlabel())
ax2.set_xlabel(r'$m_{t\bar{t}}$')
ax2.set_xticks(ax1.get_xticks())
ax2.set_xlim(800,3300)

ax1.legend(loc=1, fontsize=15)


imagefile = f'images/png/closureTest/{IOV}/closuretest_inclusive.png'

plt.savefig(imagefile)
plt.savefig(imagefile.replace('png','pdf'))
print('saving ', imagefile)
print('saving ', imagefile.replace('png','pdf'))



In [ ]:

fig, (ax1, ax2) = plt.subplots(nrows=2, height_ratios=[3, 1])

IOV = '2016'


textfontsize = 15


# hqcd = f['QCD_Pass_postfit_projy1;1']
hqcd = f['TotalBkg_Pass_postfit_projy1;1']

httbar = f['16_TTbar_Pass_postfit_projy1;1']
hdata = f['data_obs_Pass_postfit_projy1;1']


hep.histplot(hqcd, histtype='fill', color='xkcd:pale gold', label='NTMJ Bkg Est', ax=ax1)
hep.histplot(httbar, histtype='fill', color='xkcd:deep red', label='TTbar', ax=ax1)
hep.histplot(hdata, histtype='errorbar', color='k', ax=ax1, label='Data')



text = 'Preliminary'+'\n'+r'$\Delta y$ inclusive'+', '+r'b tag inclusive'+ '\n' + r'125 < $m_{t}$ < 225 GeV'
hep.cms.label('', data=True, lumi='{0:0.1f}'.format(lumi['2016']/1000.), year='2016', loc=2, fontsize=textfontsize, ax=ax1)
hep.cms.text(text, loc=2, fontsize=textfontsize, ax=ax1)




ax1.set_yscale('log')
ax1.set_ylim(1e-3,1e4)
ax1.set_xlim(800,3700)


ax1.set_ylabel('Events/bin', fontsize=20)

bkg_vals = (hqcd.values() + httbar.values())

ratio_plot = hdata.values() / bkg_vals
ratio_err = (ratio_plot)**2 * (1/hdata.values() + 1/bkg_vals)


print(ratio_plot)

ax2.errorbar(hdata.axis().centers(), ratio_plot, yerr=ratio_err, color='k', ls='', marker='o')

ax2.fill_between(hdata.axis().centers(), -1*hqcd.errors()/hqcd.values(), hqcd.errors()/hqcd.values(), color='lightgrey')

ax2.set_ylim(0,3)
ax2.axhline(1, color='black', ls='--')
ax2.set_ylabel('Data/Bkg', fontsize=20)
ax2.set_xlabel(ax1.get_xlabel())
ax2.set_xlabel(r'$m_{t\bar{t}}$')
ax2.set_xticks(ax1.get_xticks())
ax2.set_xlim(800,3700)

ax1.legend(loc=1, fontsize=textfontsize)


plt.show()

# imagefile = f'images/png/closureTest/{IOV}/closuretest_inclusive.png'

# plt.savefig(imagefile)
# plt.savefig(imagefile.replace('png','pdf'))
# print('saving ', imagefile)
# print('saving ', imagefile.replace('png','pdf'))



In [ ]:
hdata.errors()

In [ ]:
hqcd.errors()

In [ ]:
(hdata.values() - hqcd.values())/hdata.errors()

In [ ]:
def getHist(hname, ds, bkgest, year, sum_axes=['anacat'], integrate_axes={}):
    
    ######################################################################################
    # hname = histogram name (example: 'ttbarmass')                                      #
    # ds = dataset name (example: 'JetHT')                                               #
    # bkgest = boolean, True if bkg estimate applied                                     #
    # year = '2016APV' or '2016' or '2017' or '2018'                                     #
    # sum_axes = names of axes to sum over for scikit-hep/hist histogram                 #
    # integrate_axes = range to integrate over axis (example: {'anacat': [0,1,2,3,4,5]}) #
    ######################################################################################    

    
    # load histograms and get scale factors
    coffeaFiles = getCoffeaFilenames()
    
    
   
    cfiles = []
    sf = []
    bkgest_str = 'weighted' if bkgest else 'unweighted'
    
    
    try:
        
        for key, file in coffeaFiles[ds][bkgest_str][year].items():
            loaded_file = util.load(file)
            cfiles.append(loaded_file)

            if 'TTbar' in ds and '700to1000' in key:
                sf.append(1.)
#                 sf.append(lumi[year] * ttbar_xs1 * toptag_sf**2 / loaded_file['cutflow']['sumw'])
            elif 'TTbar' in ds and '1000toInf' in key:
                sf.append(1.)
#                 sf.append(lumi[year] * ttbar_xs2 * toptag_sf**2 / loaded_file['cutflow']['sumw'])     
            elif 'QCD' in ds:
                sf.append(lumi[IOV] * qcd_xs / loaded_file['cutflow']['sumw'])  
            else:
                sf.append(1.)
    except:
        
        loaded_file = util.load(coffeaFiles[ds][bkgest_str][year])
        cfiles.append(loaded_file)
        if 'TTbar' in ds and '700to1000' in key:
            sf.append(lumi[year] * ttbar_xs1 * toptag_sf**2 / loaded_file['cutflow']['sumw'])
        elif 'TTbar' in ds and '1000toInf' in key:
#             sf.append(1.)
            sf.append(lumi[year] * ttbar_xs2 * toptag_sf**2 / loaded_file['cutflow']['sumw'])     
        elif 'QCD' in ds:
            sf.append(lumi[IOV] * qcd_xs / loaded_file['cutflow']['sumw'])  
        else:
            sf.append(1.)

        
    
    # sum or integrate axes for all hists from dataset eras or pt bins
    sum_axes_dict = {ax:sum for ax in sum_axes}
    
    hists = []
    for cfile in cfiles:
            hists.append(cfile[hname][integrate_axes][sum_axes_dict])
    
    
    # sum all hists from dataset eras or pt bins
    histo = hists[0]*sf[0]
    if len(hists) > 1:
        for i in range(len(hists) - 1): 
            histo = histo + hists[i+1]*sf[i+1]
            
            
    
    return histo
    

In [ ]:
hqcd = getHist('mtt_vs_mt', 'QCD', False, IOV,
                 sum_axes=['anacat'],
                 integrate_axes={'anacat':signal_cats,'systematic':'nominal'})[{'jetmass':sum}]

In [ ]:
hqcd.plot()

In [ ]:
import coffea
import awkward as ak

In [ ]:
coffea.__version__


In [ ]:
ak.__version__


In [ ]:
nus = uproot.open('nuisance_pulls.root')

In [ ]:
nus.keys()

In [ ]:
nus['nuisances;1']

In [ ]:
nus['post_fit_errs;1']

In [ ]:
nus['asdf;1']

In [ ]:
print("""
name                                                 pre fit                        b-only fit                           s+b fit         rho  approx impact
ttbar_xsec                                   0.000000 +/- 1.000000    +12.93 +/- 0.00 (+12.93sig, 0.00)    !+3.94 +/- 0.81 (+3.94sig, 0.81)!     !-0.80!      -0.136
lumi                                         0.000000 +/- 1.000000    +1.27 +/- 0.00 (+1.27sig, 0.00)    !+0.39 +/- 1.01 (+0.39sig, 1.01)!       -0.07      -0.015
jer                                          0.000000 +/- 1.000000    +0.04 +/- 0.00 (+0.04sig, 0.00)    !+0.04 +/- 0.07 (+0.04sig, 0.07)!     !-0.02!      -0.000
pileup                                       0.000000 +/- 1.000000    -0.29 +/- 0.00 (-0.29sig, 0.00)    !+0.03 +/- 0.85 (+0.03sig, 0.85)!       +0.01      +0.002
pdf                                          0.000000 +/- 1.000000    -1.38 +/- 0.00 (-1.38sig, 0.00)    !-0.10 +/- 1.28 (-0.10sig, 1.28)!       -0.01      -0.003
jes                                          0.000000 +/- 1.000000    +0.91 +/- 0.00 (+0.91sig, 0.00)    !+0.20 +/- 0.25 (+0.20sig, 0.25)! 
""")
      
      

In [ ]:
fig, ax1 = plt.subplots(figsize=(5,7))


ymax = ax1.get_ylim()[1]
ymin = ax1.get_ylim()[0]


ax1.fill_between([-1, 1], [-1, -1], [2*ymax, 2*ymax], color='lime')
ax1.fill_between([-2, -1], [-1, -1], [2*ymax, 2*ymax], color='yellow')
ax1.fill_between([1, 2], [-1, -1], [2*ymax, 2*ymax], color='yellow')
ax1.set_xlim(-5,5)
ax1.set_ylim(-.1, ymax*1.1)



yticks = np.linspace(ymin, ymax, 6)
ylabs = ['jes', 'jer', 'pdf', 'pileup', 'ttbar_xs', 'lumi']

errs = {
    'jes':[0.25, 0],
    'jer': [0.07, -0.02],
    'pdf': [1.28, -0.01],
    'pileup': [0.85,0.01],
    'ttbar_xs': [0.81, -0.8],
    'lumi': [1.01, -0.07],
    
}

ax1.errorbar(0, yticks[0], xerr=1+np.array([errs[ylabs[0]]]).T, marker='o', color='k')
ax1.errorbar(0, yticks[1], xerr=1+np.array([errs[ylabs[1]]]).T, marker='o', color='k')
ax1.errorbar(0, yticks[2], xerr=1+np.array([errs[ylabs[2]]]).T, marker='o', color='k')
ax1.errorbar(0, yticks[3], xerr=1+np.array([errs[ylabs[3]]]).T, marker='o', color='k')
ax1.errorbar(0, yticks[4], xerr=1+np.array([errs[ylabs[4]]]).T, marker='o', color='k')
ax1.errorbar(0, yticks[5], xerr=1+np.array([errs[ylabs[5]]]).T, marker='o', color='k')


ax1.set_yticks(yticks)
ax1.set_yticklabels(ylabs)

ax1.set_xlabel('Post Fit Nuisance Parameter Value', fontsize=15, loc='center')

plt.savefig('images/nuisance.pdf')
plt.savefig('images/nuisance.png')

In [ ]:
ax1.get_ylim()[1]